In [2]:
class recommender_system(object):
    def __init__(self):
        self.prediction = None
    def readfile(filename):
        filecontents = filename.readlines()
        contents=[]
        for line in filecontents:
            foo = line.strip('\n')
            contents.append([int(n) for n in foo.split(',')])
        data=np.asarray(contents)
        return data
    def MSE(self,R,d):
        # Replace the nan values by zero
        index=np.isnan(R)
        Rnew = R.copy()
        row, col = Rnew.shape
        Rnew[index]=0
        U,s,V = LA.svd(Rnew, full_matrices=False)
        s[d:]=0
        out = np.dot(U, np.dot(np.diag(s), V))
        Rnew[index]=out[index]
        self.prediction = out
        MSE=np.linalg.norm(out-Rnew)**2
        return MSE/(col*row-sum(sum(index)))
    def MLF(self, R, d, Lambda):
        row, col= R.shape
        U =  np.random.rand(row, d)
        V = np.random.rand(d, col)
        Rnew = R.copy()
        for i in range(10):
            current = np.dot(U, V)
            U=self.updateU(Rnew,V,Lambda)
            V=self.updateV(Rnew,U,Lambda)
        out = np.dot(U,V)
        self.prediction = out
        MSE=self.rmse(out, Rnew)
        return MSE/(col*row-sum(sum(index)))
    def updateU(self, R,V,Lambda):
        U = np.linalg.solve(np.dot(V, V.T)+Lambda*np.eye(len(V)), np.dot(V, R.T))        
        return U.T
    def updateV(self, R,U,Lambda):
        V = np.linalg.solve(np.dot(U.T,U)+Lambda*np.eye(len(U[0])), np.dot(U.T, R))
        return V
    def rmse(self, prediction, ground_truth):
        prediction = prediction[ground_truth.nonzero()].toarray() 
        ground_truth = ground_truth[ground_truth.nonzero()].toarray()
        return sqrt(mean_squared_error(prediction, ground_truth))
    def validate_accuracy(self,data):
        pred = self.prediction
        N = data.shape[0]
        n = 0
        for row in data:
            if (pred[row[0]-1, row[1]-1]>0 and row[2] == 1)\
            or (pred[row[0]-1, row[1]-1]<=0 and row[2] == 0):
                n += 1
        return float(n)/N